In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.conf import SparkConf
config = SparkConf()
# config.set("property", "value")
config.setMaster("local").setAppName("StocksAssignment")

from pyspark.sql import SparkSession
# spark Session, entry point for Spark SQL, DataFrame
spark = SparkSession.builder\
                    .config(conf=config)\
                    .getOrCreate()

sc = spark.sparkContext

22/03/13 10:42:25 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.80.128 instead (on interface ens33)
22/03/13 10:42:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/13 10:42:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/13 10:42:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/03/13 10:42:30 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
sectorDf = spark.read.format("csv")\
                .option("header", True)\
                .option("inferSchema", True)\
                .load("hdfs://localhost:9000/StocksAssignment")

sectorDf.printSchema()
sectorDf.show(5)

root
 |-- _c0: integer (nullable = true)
 |-- SYMBOL: string (nullable = true)
 |-- SERIES: string (nullable = true)
 |-- OPEN: double (nullable = true)
 |-- HIGH: double (nullable = true)
 |-- LOW: double (nullable = true)
 |-- CLOSE: double (nullable = true)
 |-- LAST: double (nullable = true)
 |-- PREVCLOSE: double (nullable = true)
 |-- TOTTRDQTY: integer (nullable = true)
 |-- TOTTRDVAL: double (nullable = true)
 |-- TIMESTAMP: string (nullable = true)
 |-- TOTALTRADES: integer (nullable = true)
 |-- ISIN: string (nullable = true)
 |-- Unnamed: 13: string (nullable = true)

+---+----------+------+--------+--------+-------+--------+-------+---------+---------+--------------+-----------+-----------+------------+-----------+
|_c0|    SYMBOL|SERIES|    OPEN|    HIGH|    LOW|   CLOSE|   LAST|PREVCLOSE|TOTTRDQTY|     TOTTRDVAL|  TIMESTAMP|TOTALTRADES|        ISIN|Unnamed: 13|
+---+----------+------+--------+--------+-------+--------+-------+---------+---------+--------------+-----------

22/03/13 10:43:06 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , SYMBOL, SERIES, OPEN, HIGH, LOW, CLOSE, LAST, PREVCLOSE, TOTTRDQTY, TOTTRDVAL, TIMESTAMP, TOTALTRADES, ISIN, Unnamed: 13
 Schema: _c0, SYMBOL, SERIES, OPEN, HIGH, LOW, CLOSE, LAST, PREVCLOSE, TOTTRDQTY, TOTTRDVAL, TIMESTAMP, TOTALTRADES, ISIN, Unnamed: 13
Expected: _c0 but found: 
CSV file: hdfs://localhost:9000/StocksAssignment/March_07.csv


In [4]:
sectorDf.rdd.getNumPartitions()

1

In [21]:
# df_orders.drop('cust_no').show()
newstocksDF = sectorDf.drop('_c0','Unnamed: 13')
newstocksDF.show()

+----------+------+--------+--------+-------+--------+-------+---------+---------+--------------+-----------+-----------+------------+--------------------+--------------------+
|    SYMBOL|SERIES|    OPEN|    HIGH|    LOW|   CLOSE|   LAST|PREVCLOSE|TOTTRDQTY|     TOTTRDVAL|  TIMESTAMP|TOTALTRADES|        ISIN|                GAIN|               GAINP|
+----------+------+--------+--------+-------+--------+-------+---------+---------+--------------+-----------+-----------+------------+--------------------+--------------------+
| 20MICRONS|    EQ|   69.95|    71.0|  68.45|    69.8|   70.4|     71.6|   140616|     9823167.1|07-MAR-2022|       1742|INE144J01027|-0.15000000000000568|-0.21443888491780658|
|21STCENMGM|    EQ|   27.95|   27.95|  27.95|   27.95|  27.95|     28.5|      793|      22164.35|07-MAR-2022|         34|INE253B01015|                 0.0|                 0.0|
| 3IINFOLTD|    EQ|   52.25|    55.1|   51.0|    55.1|   55.1|     52.5|  1209245| 6.554553075E7|07-MAR-2022|      

## Finding the gain and gain%

In [22]:
# add/derive a column withColumn GAIN = CLOSE - OPEN
# add/derive a column withColumn GAINP = ( GAIN / OPEN ) * 100
# where as abs is function from pyspark.sql.functions
from pyspark.sql.functions import col
newstocksDF = newstocksDF.withColumn("GAIN", col("CLOSE") - col("OPEN"))\
                 .withColumn("GAINP", (col("GAIN") / col("OPEN") )   * 100)

newstocksDF.printSchema()
newstocksDF.show(5)

root
 |-- SYMBOL: string (nullable = true)
 |-- SERIES: string (nullable = true)
 |-- OPEN: double (nullable = true)
 |-- HIGH: double (nullable = true)
 |-- LOW: double (nullable = true)
 |-- CLOSE: double (nullable = true)
 |-- LAST: double (nullable = true)
 |-- PREVCLOSE: double (nullable = true)
 |-- TOTTRDQTY: integer (nullable = true)
 |-- TOTTRDVAL: double (nullable = true)
 |-- TIMESTAMP: string (nullable = true)
 |-- TOTALTRADES: integer (nullable = true)
 |-- ISIN: string (nullable = true)
 |-- GAIN: double (nullable = true)
 |-- GAINP: double (nullable = true)

+----------+------+--------+--------+-------+--------+-------+---------+---------+--------------+-----------+-----------+------------+--------------------+--------------------+
|    SYMBOL|SERIES|    OPEN|    HIGH|    LOW|   CLOSE|   LAST|PREVCLOSE|TOTTRDQTY|     TOTTRDVAL|  TIMESTAMP|TOTALTRADES|        ISIN|                GAIN|               GAINP|
+----------+------+--------+--------+-------+--------+-------+----

In [16]:
from pyspark.conf import SparkConf
config = SparkConf()
# config.set("property", "value")
config.setMaster("local").setAppName("StocksAssignment")

from pyspark.sql import SparkSession
sectorDf.drop('_c0','Unnamed: 13').write.mode("overwrite").parquet("hdfs://localhost:9000/stocksAssignmentparquet")

In [17]:
# READ Parquet FILE into DataFrame
# inferSchema not needed as parquet has schema itself
sectorParquetDf = spark.read.format("parquet")\
                .load("hdfs://localhost:9000/stocksAssignmentparquet")

sectorParquetDf.printSchema()
sectorParquetDf.show(5)

root
 |-- SYMBOL: string (nullable = true)
 |-- SERIES: string (nullable = true)
 |-- OPEN: double (nullable = true)
 |-- HIGH: double (nullable = true)
 |-- LOW: double (nullable = true)
 |-- CLOSE: double (nullable = true)
 |-- LAST: double (nullable = true)
 |-- PREVCLOSE: double (nullable = true)
 |-- TOTTRDQTY: integer (nullable = true)
 |-- TOTTRDVAL: double (nullable = true)
 |-- TIMESTAMP: string (nullable = true)
 |-- TOTALTRADES: integer (nullable = true)
 |-- ISIN: string (nullable = true)

+----------+------+--------+--------+-------+--------+-------+---------+---------+--------------+-----------+-----------+------------+
|    SYMBOL|SERIES|    OPEN|    HIGH|    LOW|   CLOSE|   LAST|PREVCLOSE|TOTTRDQTY|     TOTTRDVAL|  TIMESTAMP|TOTALTRADES|        ISIN|
+----------+------+--------+--------+-------+--------+-------+---------+---------+--------------+-----------+-----------+------------+
| 20MICRONS|    EQ|   69.95|    71.0|  68.45|    69.8|   70.4|     71.6|   140616|     9

In [29]:
 # lag - previous lag
from pyspark.sql.window import Window
from pyspark.sql.functions import lag, desc
windowSpec = Window.partitionBy("TIMESTAMP").orderBy("TOTALTRADES")
# df = empDf.withColumn("lag", lag("salary",1).over(windowSpec))
newdf = newstocksDF.withColumn("lag", lag("TOTALTRADES",1).over(windowSpec))
newdf.show()

+----------+------+------+------+------+------+------+---------+---------+----------+-----------+-----------+------------+----+-----+----+
|    SYMBOL|SERIES|  OPEN|  HIGH|   LOW| CLOSE|  LAST|PREVCLOSE|TOTTRDQTY| TOTTRDVAL|  TIMESTAMP|TOTALTRADES|        ISIN|GAIN|GAINP| lag|
+----------+------+------+------+------+------+------+---------+---------+----------+-----------+-----------+------------+----+-----+----+
| 563GS2026|    GS|105.39|105.39|105.39|105.39|105.39|   100.38|       10|    1053.9|07-MAR-2022|          1|IN0020210012| 0.0|  0.0|null|
| 737GS2023|    GS|100.01|100.01|100.01|100.01|100.01|   100.02|       30|    3000.3|07-MAR-2022|          1|IN0020180025| 0.0|  0.0|   1|
| 824GS2027|    GS|101.01|101.01|101.01|101.01|101.01|   101.02|       30|    3030.3|07-MAR-2022|          1|IN0020060078| 0.0|  0.0|   1|
|    ACCORD|    SM|  32.0|  32.0|  32.0|  32.0|  32.0|     31.6|     2000|   64000.0|07-MAR-2022|          1|INE113X01015| 0.0|  0.0|   1|
| AMBANIORG|    SM| 110.0| 

In [6]:
# add/derive a column withColumn GAIN = CLOSE - OPEN
# add/derive a column withColumn GAINP = ( GAIN / OPEN ) * 100
# where as abs is function from pyspark.sql.functions
from pyspark.sql.functions import col
dailyDf = dailyDf.withColumn("GAIN", col("CLOSE") - col("OPEN"))\
                 .withColumn("GAINP", (col("GAIN") / col("OPEN") )   * 100)

dailyDf.printSchema()
dailyDf.show(5)

root
 |-- SYMBOL: string (nullable = true)
 |-- SERIES: string (nullable = true)
 |-- OPEN: double (nullable = true)
 |-- HIGH: double (nullable = true)
 |-- LOW: double (nullable = true)
 |-- CLOSE: double (nullable = true)
 |-- LAST: double (nullable = true)
 |-- PREVCLOSE: double (nullable = true)
 |-- TOTTRDQTY: integer (nullable = true)
 |-- TOTTRDVAL: double (nullable = true)
 |-- TIMESTAMP: string (nullable = true)
 |-- TOTALTRADES: integer (nullable = true)
 |-- ISIN: string (nullable = true)
 |-- GAIN: double (nullable = true)
 |-- GAINP: double (nullable = true)

+----------+------+-------+-------+-------+-------+-------+---------+---------+-------------+-----------+-----------+------------+--------------------+-------------------+
|    SYMBOL|SERIES|   OPEN|   HIGH|    LOW|  CLOSE|   LAST|PREVCLOSE|TOTTRDQTY|    TOTTRDVAL|  TIMESTAMP|TOTALTRADES|        ISIN|                GAIN|              GAINP|
+----------+------+-------+-------+-------+-------+-------+---------+-------

In [8]:
# add/derive a column withColumn GAIN = CLOSE - OPEN
# add/derive a column withColumn GAINP = ( GAIN / OPEN ) * 100
# where as abs is function from pyspark.sql.functions
from pyspark.sql.functions import col
dailyDf = dailyDf.withColumn("GAIN", col("CLOSE") - col("OPEN"))\
                 .withColumn("GAINP", (col("GAIN") / col("OPEN") )   * 100)

dailyDf.printSchema()
dailyDf.show(5)

root
 |-- SYMBOL: string (nullable = true)
 |-- SERIES: string (nullable = true)
 |-- OPEN: double (nullable = true)
 |-- HIGH: double (nullable = true)
 |-- LOW: double (nullable = true)
 |-- CLOSE: double (nullable = true)
 |-- LAST: double (nullable = true)
 |-- PREVCLOSE: double (nullable = true)
 |-- TOTTRDQTY: integer (nullable = true)
 |-- TOTTRDVAL: double (nullable = true)
 |-- TIMESTAMP: string (nullable = true)
 |-- TOTALTRADES: integer (nullable = true)
 |-- ISIN: string (nullable = true)
 |-- GAIN: double (nullable = true)
 |-- GAINP: double (nullable = true)

+----------+------+-------+-------+-------+-------+-------+---------+---------+-------------+-----------+-----------+------------+--------------------+-------------------+
|    SYMBOL|SERIES|   OPEN|   HIGH|    LOW|  CLOSE|   LAST|PREVCLOSE|TOTTRDQTY|    TOTTRDVAL|  TIMESTAMP|TOTALTRADES|        ISIN|                GAIN|              GAINP|
+----------+------+-------+-------+-------+-------+-------+---------+-------